In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import scanpy as sc
import pandas as pd
import newick
from Bio import Phylo
from anndata import AnnData
import ete3
from scipy.sparse import issparse
import os
import sys
import pickle as pkl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lineageot.inference as sim_inf

#import utils
from jax import numpy as jnp
import ot
import lineageot
from collections import defaultdict
import networkx as nx

In [4]:
from moscot.framework.estimators import LineageEstimator
import sys
sys.path.append("../..")
from moscot.framework import utils_tmp

In [5]:
data_dir = "/Users/DominikKlein/Documents/PhD/data/tedsim_zoe/adatas/adata_pa0.8_ss0.6.h5ad"

In [6]:
adata = sc.read(data_dir)

Only considering the two last: ['.6', '.h5ad'].
Only considering the two last: ['.6', '.h5ad'].


In [6]:
tree = newick.loads(adata.uns['tree'])[0]

In [7]:
adatas = utils_tmp.create_adatas(adata, 6)

In [9]:
dep = 6
tree = newick.loads(adatas['orig'].uns['tree'])[0]
G = utils_tmp.gt_tree(adatas['processed'], tree, create_cell_bool=False)
d = utils_tmp.add_node_depths(adatas['processed'], G)

top, bottom = utils_tmp.cut_tree_at_depth(G, dep)

In [11]:
bottom_leaves = sim_inf.get_leaves(bottom, include_root=False)
top_leaves = sim_inf.get_leaves(top, include_root=False)
top_cell_ids = [top.nodes[n]['cellID'] for n in top_leaves]
#assert all(top.nodes[n]['node_depth'] == d for n in top_leaves)
adata_tmp = adata.copy()
utils_tmp.process(adata)
adata_bottom = adata_tmp[:1024].copy()
adata_top = adata_tmp[adata_tmp.obs['cellID'].isin(top_cell_ids)][[str(c) for c in top_cell_ids]].copy()
jdata = adata_top.concatenate(adata_bottom, batch_key='time_point')

In [12]:
utils_tmp.process(adata_bottom)
utils_tmp.process(adata_top)
utils_tmp.process(jdata)

In [13]:
jdata.obs.time_point = jdata.obs.time_point.astype(int, copy=False)

# Use Moscot Framework

In [14]:
key = "time_point"
policy = [(0,1)]

In [15]:
estimator = LineageEstimator(jdata, {0:top, 1:bottom})

In [16]:
estimator.prepare(key, policy)

LineageEstimator(adata=AnnData object with n_obs × n_vars = 1088 × 500
    obs: 'parent', 'cluster', 'depth', 'cellID', 'node_depth', 'time_point'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'umap'
    obsm: 'barcodes', 'X_pca', 'X_umap', 'X_pca_50'
    varm: 'PCs'
    obsp: 'distances', 'connectivities',
                 tree_dict={0: <networkx.classes.digraph.DiGraph object at 0x7faa3acf3580>,
                            1: <networkx.classes.digraph.DiGraph object at 0x7faa3bab65b0>})

In [17]:
epsilon = 0.5
alpha = 0.5
estimator.fit(epsilon, alpha)

It.  |Loss        |Rel. loss   |Abs. loss   |tau         |converged   |eps         
-----------------------------------------------------------------------------------
    1|4.213724e+02|1.000000e+00|4.213724e+02|0.000000e+00|           1|5.000000e-01
    2|4.213724e+02|0.000000e+00|0.000000e+00|0.000000e+00|           1|5.000000e-01
